In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Let's start by gathering the MCP params for our trader

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [3]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [4]:
researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
]

### Now create the MCPServerStdio for each

In [5]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

In [6]:
mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [7]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [8]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [10]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



Here is the latest summary on Amazon based on current data:

- Amazon's stock is trading around $225.34, down about 1.6% recently, with a 52-week range from $161.38 to $242.52.
- The market cap stands at approximately $2.4 trillion and the price-to-earnings (P/E) ratio is about 34.4.
- Earnings are expected on October 29-30, 2025.
- Amazon operates across retail (consumer products), advertising, subscription services, and cloud computing (AWS), with a solid revenue base exceeding $670 billion (TTM).
- Some recent market dynamics show a mild decline consistent with broader market pressures like treasury yield jumps and tariff/Fed uncertainties.
- Key operational highlights: strong gross margin (~49.6%), net margin (~10.5%), and high return on equity (~25%).
- There are mentions of AWS cloud expansion efforts and ongoing e-commerce business strategies.

Unfortunately, detailed latest news articles were inaccessible at the moment, but financial indicators suggest Amazon is navigating moderate volatility and focusing on growth areas like cloud services.

Would you like me to monitor for breaking news or deeper insights into any specific segment?

### Look at the trace

https://platform.openai.com/traces

In [12]:
ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Vino").reset(ed_initial_strategy)

display(Markdown(await read_accounts_resource("Vino")))
display(Markdown(await read_strategy_resource("Vino")))

{"name": "vino", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-09-03 15:09:43", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [13]:
agent_name = "Vino"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [14]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Vino and your account is under your name, Vino.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "vino", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-09-03 15:09:43", 10000.0], ["2025-09-03 15:14:21", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.


### And to run our Trader

In [15]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

### Summary of Actions:

1. **Market Analysis**: 
   - Investigated current stock market trends, identifying strong movements in tech stocks, particularly Apple (AAPL) and Tesla (TSLA). Noted Amazon (AMZN) and Netflix (NFLX) experiencing downward pressure post-earnings.

2. **Stock Price Checks**: 
   - Prices at the time of my analysis:
     - AAPL: $230.18
     - TSLA: $330.02
     - AMZN: $225.34
     - NFLX: $1214.11

3. **Trades Executed**: 
   - **Buys**:
     - Purchased **20 shares of AAPL** at $230.18 based on positive momentum after recent earnings.
     - Purchased **10 shares of TSLA** at $330.02, anticipating a rebound due to upcoming delivery numbers.
   
   - **Sells** (Attempted):
     - Attempted to **sell 5 shares of AMZN** but faced an error due to insufficient shares.
     - Attempted to **sell 3 shares of NFLX** but faced an error due to insufficient shares.

### Current Portfolio Status:
- **Holdings**:
  - AAPL: 20 shares
  - TSLA: 10 shares
- **Cash Balance**: $2,096.22
- **Total Portfolio Value**: $9,984.22
- **Total Profit/Loss**: -$15.78

### Next Steps:
- Consider additional trades with remaining cash.
- Monitor the performance of AAPL and TSLA for potential price movements.
- Investigate further trading opportunities in the market, especially focusing on the tech sector and energy stocks. 

Would you like me to continue searching for new opportunities or execute more trades?

### Then go and look at the trace

http://platform.openai.com/traces


In [16]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "vino", "balance": 2096.224, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"AAPL": 20, "TSLA": 10}, "transactions": [{"symbol": "AAPL", "quantity": 20, "price": 230.17944, "timestamp": "2025-09-03 15:15:33", "rationale": "AAPL is experiencing positive momentum after recent earnings reports. The tech sector is moving along with strong overall market sentiment."}, {"symbol": "TSLA", "quantity": 10, "price": 330.01872000000003, "timestamp": "2025-09-03 15:15:33", "rationale": "TSLA shows potential for a rebound with upcoming delivery numbers and market buzz surrounding their new models."}], "portfolio_value_time_series": [["2025-09-03 15:09:43", 10000.0], ["2025-09-03 15:14:21", 10000.0], ["2025-09-03 15:15:33", 9990.8112], ["2025-09-03 15:15:33", 9984.224], ["2025-09-03 15:18:06", 9984.224]], "total_portfolio_value": 9984.224, "total_profit_loss": -15.77599999999984}'

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [ ]:
from traders import Trader


In [ ]:
trader = Trader("Vino")

In [ ]:
await trader.run()

In [ ]:
await read_accounts_resource("Ed")

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [ ]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")